<a href="https://colab.research.google.com/github/katrinag2004/ds2002assignments/blob/main/financeapi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import requests
import pandas as pd
import json

# Your API key
apikey = "uWYBqhtqtN2OZSAnQ6b54Hhuc3sPlOA5niRUY59e"

def fetch_stock_data(symbols):
    url = "https://yfapi.net/v6/finance/quote"
    querystring = {"symbols": symbols}
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    # Log the response for debugging
    print("Stock Data Response:", json.dumps(data, indent=2))

    stock_data = []
    if 'quoteResponse' in data and 'result' in data['quoteResponse']:
        for stock in data['quoteResponse']['result']:
            stock_data.append({
                'Stock Ticker': stock.get('symbol', 'N/A'),
                'Company': stock.get('longName', 'N/A'),
                'Current Market Price': stock.get('regularMarketPrice', 'N/A')
            })
    else:
        print("Error: No stock data found.")
    return stock_data


def fetch_additional_data(symbol, module):
    url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    querystring = {"modules": module}
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    # Log the response for debugging
    print("Additional Data Response:", json.dumps(data, indent=2))

    if 'quoteSummary' in data and 'result' in data['quoteSummary']:
        return data['quoteSummary']['result'][0].get(module, {})
    else:
        print("Error: Could not retrieve additional data.")
        return {}


def fetch_trending_stocks():
    url = "https://yfapi.net/v1/finance/trending/US"
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers)
    data = response.json()

    # Log the response for debugging
    print("Trending Stocks Response:", json.dumps(data, indent=2))

    trending_stocks = []
    if 'finance' in data and 'result' in data['finance'] and data['finance']['result']:
        for stock in data['finance']['result'][0]['quotes']:
            trending_stocks.append({
                'Stock Ticker': stock.get('symbol', 'N/A'),
                'Company': stock.get('shortName', stock.get('longName', 'N/A')),
                'Current Market Price': stock.get('regularMarketPrice', 'N/A'),
                '52 Week High': stock.get('fiftyTwoWeekHigh', 'N/A'),
                '52 Week Low': stock.get('fiftyTwoWeekLow', 'N/A')
            })
    else:
        print("Error: No trending stocks data found.")
    return trending_stocks


if __name__ == "__main__":
    # Task 1: Fetch Basic Stock Data
    symbols = input("Enter stock symbols separated by commas: ")
    stock_data = fetch_stock_data(symbols)
    if stock_data:
        for stock in stock_data:
            price = stock['Current Market Price']
            price_display = f"${price:.2f}" if isinstance(price, (int, float)) else price
            print(f"Stock Ticker: {stock['Stock Ticker']}, Company: {stock['Company']}, Current Market Price: {price_display}")

    # Task 2.1: Fetch Additional Data Using Modules
    symbol = input("Enter a stock symbol for additional data: ")
    module = input("Choose a module (summaryDetail, financialData, defaultKeyStatistics): ")
    additional_data = fetch_additional_data(symbol, module)
    if additional_data:
        df = pd.DataFrame([additional_data])
        print(df)

    # Task 2.2: Fetch Current Trending Stocks
    trending_stocks = fetch_trending_stocks()
    if trending_stocks:
        df = pd.DataFrame(trending_stocks)
        print(df)



Enter stock symbols separated by commas: AAPL,GOOG
Stock Data Response: {
  "quoteResponse": {
    "result": [
      {
        "language": "en-US",
        "region": "US",
        "quoteType": "EQUITY",
        "typeDisp": "Equity",
        "quoteSourceName": "Nasdaq Real Time Price",
        "triggerable": true,
        "customPriceAlertConfidence": "HIGH",
        "currency": "USD",
        "regularMarketChangePercent": -2.768781,
        "regularMarketPrice": 240.2,
        "corporateActions": [],
        "gmtOffSetMilliseconds": -18000000,
        "market": "us_market",
        "esgPopulated": false,
        "regularMarketTime": 1740599102,
        "exchange": "NMS",
        "messageBoardId": "finmb_24937",
        "exchangeTimezoneName": "America/New_York",
        "exchangeTimezoneShortName": "EST",
        "shortName": "Apple Inc.",
        "longName": "Apple Inc.",
        "tradeable": false,
        "cryptoTradeable": false,
        "marketState": "REGULAR",
        "hasPrePos